In [1]:
import ocean_grid_generator as ogg
import numpy

In [2]:
nj, ni = 1152, 1440
jEq = 527
Re = 6378e3

In [3]:
# This creates a NH mercator between lat=0 and lat=~68.1
xmn,ymn = ogg.generate_mercator_grid(1440, 0, 68.1, -300, 360, 1, shift_equator_to_u_point=False, ensure_nj_even=False)
print('Northern latitude of OGG Mercator        = ', ymn[-1,0])

Requesting Mercator grid with phi range: phi_s,phi_n= 0 68.1
   y*= [  0 376] nj= 377
  *Equator may not be a u-point!
   Generating Mercator grid with phi range: phi_s,phi_n= [ 0.         68.05725377]
   Equator is at j= 0
  *Equator is not going to be a u-point of this grid patch.
   Final Mercator grid range= 0.0 68.05725376601046
   number of js= 377
Northern latitude of OGG Mercator        =  68.05725376601046


In [4]:
# Equatorial band
xeb,yeb = ogg.generate_latlon_grid(1440, 39, -300., 360., 0., 5., ensure_nj_even=False)

Generating regular lat-lon grid between latitudes  0.0 5.0
   generated regular lat-lon grid between latitudes  0.0 5.0
   number of js= 40


In [5]:
# Cubic band
# Note: we know that this is using extrapolated data and should really be a quartic.
N,jmatch = 130,124
ycub = ogg.lagrange_interp4( [0,1,N-2,N-1], [5,5+.13, ymn[jmatch,0], ymn[jmatch+1,0]], numpy.arange(0,N) )

In [6]:
# Southen spherical band
xss,yss = ogg.generate_latlon_grid(1440, 108, -300., 360., -78., 78 - ymn[-1,0], ensure_nj_even=False)
xss,yss = xss[:-1,0],yss[:-1,0]

Generating regular lat-lon grid between latitudes  -78.0 -68.05725376601046
   generated regular lat-lon grid between latitudes  -78.0 -68.05725376601046
   number of js= 109


In [7]:
# Join latitudes for Equatorial band, cubic and mercator (in northern hemisphere only)
ytmp = numpy.concatenate( (yeb[:,0], ycub[1:-1], ymn[jmatch+1:,0]) ) # 1D array
# Stitch together all grids with uniform zonal resolution (i.e. all but the Northern bi-polar cap)
yreg = numpy.concatenate( (yss, -ytmp[::-1], ytmp[1:388]) )
xreg = xeb[0,:] # 1D array (all longitudes across patches are the same)

In [8]:
# Metrics for regular grid
dxreg,dyreg,areareg = ogg.spherical_metrics(xreg, yreg, Re=Re)

In [9]:
# Bi-polar cap
xbp,ybp,dxbp,dybp = ogg.generate_bipolar_cap_mesh(1440, 238, yreg[-1], -300.)#, ensure_nj_even=False)
print( 'xbp.shape,ybp.shape =', ybp.shape, ybp.shape)

Generating bipolar grid bounded at latitude  64.86719305598032
   number of js= 239
xbp.shape,ybp.shape = (239, 1441) (239, 1441)


In [10]:
# Metrics for bi-polar cap
dxbp,dybp,areabp = ogg.numerical_metrics(xbp, ybp, Re=Re)

In [11]:
# Render the regular grid into a 2D mesh
lon,lat = numpy.meshgrid(xreg, yreg)
# Join it all together
lon = numpy.concatenate( (lon, xbp[1:,:]) )
lat = numpy.concatenate( (lat, ybp[1:,:]) )
print('lon.shape, lat.shape =', lon.shape, lat.shape)
dx = numpy.concatenate( (dxreg, dxbp[1:,:]))
dy = numpy.concatenate( (dyreg, dybp))
area = numpy.concatenate( (areareg, areabp))
angle = ogg.angle_x(lon, lat)
print('dx.shape, dy.shape, area.shape, angle.shape =', dx.shape, dy.shape, area.shape, angle.shape)

lon.shape, lat.shape = (1153, 1441) (1153, 1441)
dx.shape, dy.shape, area.shape, angle.shape = (1153, 1440) (1152, 1441) (1152, 1440) (1153, 1441)


In [12]:
ogg.write_nc(lon, lat, dx, dy, area, angle, fnam="ocean_hgrid_OM4p5.nc", no_changing_meta=True)

   Writing netcdf file with ny,nx=  1152 1440
